In [1]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import cifar10

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [3]:
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


In [4]:
# model = keras.Sequential([
#     layers.Input(shape=(32, 32, 3)),
#     layers.Conv2D(32, 3, padding='same', kernel_regularizer=regularizers.L2(0.01)),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.MaxPool2D(pool_size=(2,2)),
    
#     layers.Conv2D(64, 3, padding='same', kernel_regularizer=regularizers.L2(0.01)),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.MaxPool2D(pool_size=(2,2)),
    
#     layers.Conv2D(128, 3, padding='same', kernel_regularizer=regularizers.L2(0.01)),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
    
    # layers.Flatten(),
    # layers.Dense(64, activation='relu', kernel_regularizer=regularizers.L2(0.01)),
    # # layers.Dropout(0.5),
    # layers.Dense(10, activation='softmax')
# ])

class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size=kernel_size, padding='same')
        self.batchNorm = layers.BatchNormalization()
        self.maxPool = layers.MaxPool2D()

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.batchNorm(x)
        x = tf.nn.relu(x)
        x = self.maxPool(x)
        return x

model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    CNNBlock(32),
    CNNBlock(64),
    CNNBlock(128),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])
model.summary()

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-01-02 23:23:17.219558: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-02 23:23:17.219783: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cnn_block (CNNBlock)        (None, 16, 16, 32)        1024      
                                                                 
 cnn_block_1 (CNNBlock)      (None, 8, 8, 64)          18752     
                                                                 
 cnn_block_2 (CNNBlock)      (None, 4, 4, 128)         74368     
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 64)                131136    
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 225,930
Trainable params: 225,482
Non-trai

In [5]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/10


2023-01-02 23:23:19.822281: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-02 23:23:21.400279: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 - 41s - loss: 1.2846 - accuracy: 0.5405 - 41s/epoch - 52ms/step
Epoch 2/10
782/782 - 28s - loss: 0.8771 - accuracy: 0.6930 - 28s/epoch - 36ms/step
Epoch 3/10
782/782 - 31s - loss: 0.7318 - accuracy: 0.7449 - 31s/epoch - 39ms/step
Epoch 4/10
782/782 - 35s - loss: 0.6406 - accuracy: 0.7771 - 35s/epoch - 45ms/step
Epoch 5/10
782/782 - 36s - loss: 0.5619 - accuracy: 0.8035 - 36s/epoch - 47ms/step
Epoch 6/10
782/782 - 26s - loss: 0.5052 - accuracy: 0.8244 - 26s/epoch - 34ms/step
Epoch 7/10
782/782 - 33s - loss: 0.4409 - accuracy: 0.8460 - 33s/epoch - 42ms/step
Epoch 8/10
782/782 - 34s - loss: 0.3810 - accuracy: 0.8678 - 34s/epoch - 44ms/step
Epoch 9/10
782/782 - 32s - loss: 0.3429 - accuracy: 0.8791 - 32s/epoch - 40ms/step
Epoch 10/10
782/782 - 30s - loss: 0.2937 - accuracy: 0.8971 - 30s/epoch - 39ms/step


2023-01-02 23:28:46.828390: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


157/157 - 3s - loss: 0.7871 - accuracy: 0.7616 - 3s/epoch - 18ms/step


[0.7871312499046326, 0.7616000175476074]